In [ ]:
import matplotlib.pyplot as plt
from preprocess_functions import preprocess
import numpy as np
import os

In [ ]:
paths = []
rootdir = '/Users/rancze/Documents/Data/vestVR/Cohort2_test_Photometry_only'
#rootdir = '/Users/rancze/Documents/Data/vestVR/Cohort1/VestibularMismatch_day1'

for root, dirs, files in os.walk(rootdir):
    for filename in files:
        if filename == 'Fluorescence-unaligned.csv':
            # Get the folder one level higher
            parent_folder = os.path.dirname(root)
            paths.append(parent_folder)

# Print all the paths
[print(path) for path in paths] 

sensors = {'470':'g5-HT3', '560':'rG1', '410':'g5-HT3'}
target_area = ('X') #ASSUMES the same target area for all folders, DANGER
filtmethod = "auto"
detrendmethod = "divisive"

In [ ]:
for path in paths: 
    processed = preprocess(path, sensors)
    processed.info = processed.get_info()
    (
        processed.rawdata, 
        processed.data, 
        processed.data_seconds, 
        processed.signals, 
    ) = processed.create_basic(
        cutstart = False,
        cutend = False,
        target_area = target_area
    )
    #processed.events = processed.extract_events()
    processed.filtered = processed.low_pass_filt(method = filtmethod)
    processed.data_detrended, processed.exp_fits = processed.detrend(method = detrendmethod)
    processed.motion_corr = processed.movement_correct() #results of this are generally not used 
    processed.zscored = processed.z_score(motion = False)
    processed.deltaF_F = processed.get_deltaF_F(motion = False)
    processed.plot_all_signals()
    processed.info_csv = processed.write_info_csv()
    processed.data_csv = processed.write_preprocessed_csv()#Events = True) #optional: Events = True, motion_correct = True